In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM, MT5ForConditionalGeneration


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer_mT5_m2o = AutoTokenizer.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")
decoder_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")

tokenizer_indicbart = AutoTokenizer.from_pretrained("ai4bharat/IndicBART") #"aashay96/indic-gpt"
encoder_model =  AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
encoder_model2 = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")
decoder_model2 = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_m2o_hindi_crossSum")

In [4]:
# I want to pass a tokenizer output to the encoder_model2 model's encoder only
import torch
x = torch.tensor(tokenizer_indicbart.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
print(x.shape)
out = encoder_model2(input_ids=x)
print(out.logits.shape)
print(out.encoder_last_hidden_state.shape)


x = torch.tensor(tokenizer_mT5_m2o.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
print(x.shape)
out2 = decoder_model2.decoder(x)
summary = decoder_model2.generate(x)
print(summary, summary.shape)

torch.Size([1, 29])
torch.Size([1, 29, 64014])
torch.Size([1, 29, 1024])
torch.Size([1, 42])
tensor([[250021,    259,  13340,    259,  24954,  32949,  42365,    259,  53231,
          15805,   2513,   7397,    830,   1676,    259,  21726,  10094,   1114,
            259,  24794,   1053,  11287,   1163,    259,  23272,   8949,   6904,
            259,  17494,   1248,   1462,   1871,   1896,    260,      1]]) torch.Size([1, 35])


In [5]:
out3 = decoder_model2.lm_head(out2.last_hidden_state)

In [6]:
out2.last_hidden_state.shape, out3.shape

(torch.Size([1, 42, 768]), torch.Size([1, 42, 250112]))

In [7]:
out3.softmax(-1).argmax(-1)

tensor([[   264,   3028,  50575,    259,  13340,    975,   1676,  25659,  12604,
          42365,   1462,  53231,  15805,    975,   1890,  52835,  35557,   1053,
            259,   1676,    259,    277,    259,   1114,    259,  24794,    975,
            259,    975,    259,    259,   4585,   1890,   8559,  40150,   8559,
           3689,   3689,    830, 155486,      1,      0]])

In [8]:
summary2 = tokenizer_mT5_m2o.decode(summary[0], skip_special_tokens=True)
summary2

'कांग्रेस अध्यक्ष राहुल गांधी ने कहा है कि भारतीय जनता पार्टी को देश का निर्माण कार्य दो साल हो गए हैं.'

In [9]:
encoder_model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [19]:
decoder_model.decoder

MT5Stack(
  (embed_tokens): Embedding(250112, 768)
  (block): ModuleList(
    (0): MT5Block(
      (layer): ModuleList(
        (0): MT5LayerSelfAttention(
          (SelfAttention): MT5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768, bias=False)
            (relative_attention_bias): Embedding(32, 12)
          )
          (layer_norm): MT5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): MT5LayerCrossAttention(
          (EncDecAttention): MT5Attention(
            (q): Linear(in_features=768, out_features=768, bias=False)
            (k): Linear(in_features=768, out_features=768, bias=False)
            (v): Linear(in_features=768, out_features=768, bias=False)
            (o): Linear(in_features=768, out_features=768

In [25]:
import torch.nn as nn

class EncoderDecoderModel(nn.Module):
    def __init__(self, encoder_model, decoder_model) -> None:
        super(EncoderDecoderModel, self).__init__()
        self.shared = encoder_model.model.shared  
        self.encoder = encoder_model.model.encoder
        self.change_features_dim = nn.Linear(1024, 768)
        self.decoder = nn.Sequential(
            decoder_model.decoder.block,
            decoder_model.decoder.final_layer_norm,
            decoder_model.decoder.dropout
        )
        self.lm_head = decoder_model.lm_head
    
    def forward(self, x):
        # x = self.shared(x)
        x = self.encoder(x).last_hidden_state
        x = self.change_features_dim(x)
        x = self.decoder(x)
        x = self.lm_head(x)
        return x
        

m = EncoderDecoderModel(encoder_model2, decoder_model2)
x = torch.tensor(tokenizer_indicbart.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
print(x.shape)
out = m(x)
print(out.shape)

# out = encoder_model2(input_ids=x)
# print(out.logits.shape)
# print(out.encoder_last_hidden_state.shape)


# x = torch.tensor(tokenizer_mT5_m2o.encode("भारतीय राष्ट्रीय कांग्रेस के अध्यक्ष राहुल गांधी ने शनिवार को कहा कि भारतीय जनता पार्टी ने देश को एक दशक में वापस ले जाया ह।", return_tensors="pt"))
# print(x.shape, x)
# out2 = decoder_model2.decoder(x)
# summary = decoder_model2.generate(x)
# print(summary, summary.shape)


torch.Size([1, 29])


NotImplementedError: Module [ModuleList] is missing the required "forward" function

In [82]:
decoder_model2

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [41]:
decoder_model2

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
         

In [ ]:
decoder_model2.generate(x, max_length=100, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)